In [1]:
import os
import pandas as pd
import scipy.io as sio
import numpy as np
from tqdm import tqdm

In [2]:
record_base_path = './CPSC2018'
#def get_file(record_base_path):
files = []
for filename in os.listdir(record_base_path):
    if filename.endswith('.mat'):
        path = os.path.join(record_base_path, filename)
        files.append(path)
#return files

In [3]:
data_12 = sio.loadmat(path)['ECG'][0][0][2]
print(data_12.shape)

(12, 18488)


In [4]:
#def get_data(path):
min_batch = []

filename = []
for i in tqdm(range(len(files))):
    jieduan = data_12[:,i:i+3000]
    if jieduan.shape[1] <3000:
         jieduan = data_12[:,-3000:]
    min_batch.append(jieduan[[1,2,6,7,8,9,10,11],:].T)
    filename.append(path.split('\\')[-1][:-4])

#return np.array(min_batch).reshape(-1,3000,8),list(set(filename))[0]

100%|███████████████████████████████████████████████████████████████████████████| 6877/6877 [00:00<00:00, 16895.40it/s]


In [5]:
X = np.array(min_batch).reshape(-1,3000,8)

In [6]:
range(len(files))

range(0, 6877)

In [7]:
df = pd.read_csv('./CPSC2018/REFERENCE.csv')
#labels = df["First_label"][:1000]
y = pd.get_dummies(df["First_label"][:6877])
y

,1,2,3,4,5,6,7,8,9
0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
6872,0,0,1,0,0,0,0,0,0
6873,0,0,0,0,0,0,0,1,0
6874,0,0,0,1,0,0,0,0,0
6875,0,1,0,0,0,0,0,0,0


In [8]:
print (X.shape, y.shape)

(6877, 3000, 8) (6877, 9)


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [10]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(5501, 3000, 8) (1376, 3000, 8)
(5501, 9) (1376, 9)


In [11]:
X_train = X_train.reshape(-1, 3000, 8)
print(X_train.shape)

(5501, 3000, 8)


In [12]:
# importing the libraries
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [13]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    fig = plt.figure(figsize=(16,10))
    
    ## Loss
    fig.add_subplot(2,1,1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    fig.add_subplot(2,1,2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('training_history.png')
    plt.show()
    

In [14]:
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = models.Sequential()

#CNN1D model
model.add(layers.Conv1D(filters=64, kernel_size=6, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(3), strides=(2), padding="same"))

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(2), strides=(2), padding="same"))

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(2), strides=(2), padding="same"))

model.add(layers.Dropout(0.5))


model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(2), strides=(2), padding="same"))

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(2), strides=(2), padding="same"))

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(2), strides=(2), padding="same"))

model.add(layers.Dropout(0.5))

#LSTM model
#model.add(layers.LSTM(32, return_sequences=True))
#model.add(layers.Dropout(0.2))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(y_train.shape[1], activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2995, 64)          3136      
_________________________________________________________________
batch_normalization (BatchNo (None, 2995, 64)          256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1498, 64)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1496, 64)          12352     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1496, 64)          256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 748, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 746, 64)           1

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(2), strides=(2), padding="same"))

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(2), strides=(2), padding="same"))

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool1D(pool_size=(2), strides=(2), padding="same"))

model.add(layers.Dropout(0.5))


X_train = model.predict(X_train)
X_test = model.predict(X_test)


print(X_train.shape, X_test.shape)

# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max
print(X_train.shape, X_test.shape)

#Output
model = models.Sequential()
#model.add(layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(64))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(32))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(y_train.shape[1], activation='softmax'))


In [15]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=8),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [16]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/100
172/172 [==============================] - 48s 272ms/step - loss: 2.3242 - accuracy: 0.1799 - val_loss: 2.1229 - val_accuracy: 0.1948
Epoch 2/100
172/172 [==============================] - 45s 262ms/step - loss: 2.0620 - accuracy: 0.2464 - val_loss: 2.0695 - val_accuracy: 0.2340
Epoch 3/100
172/172 [==============================] - 44s 257ms/step - loss: 2.0418 - accuracy: 0.2460 - val_loss: 2.0557 - val_accuracy: 0.2318
Epoch 4/100
172/172 [==============================] - 47s 273ms/step - loss: 2.0322 - accuracy: 0.2483 - val_loss: 2.0536 - val_accuracy: 0.2398
Epoch 5/100
172/172 [==============================] - 46s 270ms/step - loss: 2.0126 - accuracy: 0.2603 - val_loss: 2.0555 - val_accuracy: 0.2391
Epoch 6/100
172/172 [==============================] - 48s 280ms/step - loss: 2.0283 - accuracy: 0.2567 - val_loss: 2.0656 - val_accuracy: 0.2340
Epoch 7/100
172/172 [==============================] - 46s 266ms/step - loss: 2.0264 - accuracy: 0.2473 - val_loss: 2.0548 -

KeyboardInterrupt: 

In [ ]:
plot_history(history)

In [ ]:
hist_df = pd.DataFrame(history.history) 

In [ ]:
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
# loading the trained weights
model.load_weights("best_model.hdf5")

In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
